In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
import tensorflow
from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import Input
from keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder

tensorflow.random.set_seed(2)
np.random.seed(1907)



# load dataset
df = pd.read_csv("FullData.csv")

df.head()

In [ ]:
df= df.drop('Name', axis = 1)
df= df.drop('Nationality', axis = 1)
df= df.drop('National_Position', axis = 1)
df= df.drop('National_Kit', axis = 1)
df= df.drop('Club_Kit', axis = 1)
df= df.drop('Club_Joining', axis = 1)
df= df.drop('Club', axis = 1)
df= df.drop('Height', axis = 1)
df= df.drop('Weight', axis = 1)
df= df.drop('Birth_Date', axis = 1)
df= df.drop('Contract_Expiry', axis = 1)
df= df.drop('Preffered_Position', axis = 1)

In [ ]:
df = df.dropna()
df.shape
dfCopy= df.copy()

In [ ]:
lb_make = LabelEncoder()
df['Club_Position'] = lb_make.fit_transform(df['Club_Position'])
labels = np.array(df['Club_Position'])
df = pd.get_dummies(df)


from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df, labels, test_size = 0.25, random_state = 42)
df.shape

In [ ]:
model = models.Sequential()
model.add(layers.Dense(70, activation='relu', input_shape=(train_x.shape[1],)))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(29, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


train_yC = to_categorical(train_y)
test_yC = to_categorical(test_y)

# history = model.fit(train_x, train_yC, epochs=200, batch_size=32,  verbose=0, validation_split=0.25)

best_weights = ModelCheckpoint('best.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='min')
callbacks_list = [best_weights]
history = model.fit(train_x, train_yC,epochs=10, batch_size=32,  callbacks=callbacks_list, verbose=0, validation_split=0.25)
model.load_weights("best.h5")#loads the best model saved wrt to validation loss during training

nn_pred_train = model.predict_classes(train_x)
nn_pred_test= model.predict_classes(test_x)


print ("NN Train Accuracy: ", metrics.accuracy_score(train_y, nn_pred_train))
print ("NN Test Accuracy: ", metrics.accuracy_score(test_y, nn_pred_test))
print ("NN Test MSE: ", metrics.mean_squared_error(test_y, nn_pred_test))

In [ ]:
dfCopy

In [ ]:
lb_make = LabelEncoder()
dfCopy['Club_Position'] = lb_make.fit_transform(dfCopy['Club_Position'])
labels = np.array(dfCopy['Club_Position'])

dfGK = dfCopy[['GK_Positioning', 'GK_Diving', 'GK_Kicking', 'GK_Handling', 'GK_Reflexes']]
dfDef = dfCopy[['Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Interceptions']]
dfFor = dfCopy[['Shot_Power', 'Finishing', 'Long_Shots', 'Freekick_Accuracy', 'Penalties', 'Volleys', 'Jumping', 'Heading']]
dfDefMid = dfCopy[['Speed', 'Stamina', 'Strength', 'Acceleration', 'Balance', 'Agility']]
dfMid = dfCopy[['Composure','Skill_Moves', 'Ball_Control', 'Dribbling', 'Reactions', 'Attacking_Position', 'Vision', 'Curve', 'Short_Pass', 'Long_Pass', 'Crossing']]
dfRem = dfCopy[['Rating', 'Preffered_Foot', 'Age', 'Work_Rate', 'Weak_foot']]

dfGK = pd.get_dummies(dfGK)
dfDef = pd.get_dummies(dfDef)
dfFor = pd.get_dummies(dfFor)
dfDefMid = pd.get_dummies(dfDefMid)
dfMid = pd.get_dummies(dfMid)
dfRem = pd.get_dummies(dfRem)


from sklearn.model_selection import train_test_split
train_xGK, test_xGK, train_xDef, test_xDef, train_xFor, test_xFor, train_xDefMid, \
test_xDefMid, train_xMid, test_xMid, train_xRem, \
test_xRem, train_y, test_y = train_test_split(dfGK, dfDef, dfFor, dfDefMid,dfMid, dfRem, labels, test_size = 0.25, random_state = 42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_xGK)
train_xGK = scaler.transform(train_xGK)
test_xGK = scaler.transform(test_xGK)

scalerDef = StandardScaler()
scalerDef.fit(train_xDef)
train_xDef = scalerDef.transform(train_xDef)
test_xDef = scalerDef.transform(test_xDef)

scalerFor = StandardScaler()
scalerFor.fit(train_xFor)
train_xFor = scalerFor.transform(train_xFor)
test_xFor = scalerFor.transform(test_xFor)

scalerDefMid = StandardScaler()
scalerDefMid.fit(train_xDefMid)
train_xDefMid = scalerDefMid.transform(train_xDefMid)
test_xDefMid = scalerDefMid.transform(test_xDefMid)

scalerMid = StandardScaler()
scalerMid.fit(train_xMid)
train_xMid = scalerMid.transform(train_xMid)
test_xMid = scalerMid.transform(test_xMid)

scalerRem = StandardScaler()
scalerRem.fit(train_xRem)
train_xRem = scalerRem.transform(train_xRem)
test_xRem = scalerRem.transform(test_xRem)


In [ ]:
train_xRem.shape

In [ ]:
GKInput = Input(shape=(train_xGK.shape[1],))
DefInput = Input(shape=(train_xDef.shape[1],))
ForInput = Input(shape=(train_xFor.shape[1],))
DefMidInput = Input(shape=(train_xDefMid.shape[1],))
MidInput = Input(shape=(train_xMid.shape[1],))
RemInput = Input(shape=(train_xRem.shape[1],))

GKLayer = layers.Dense(10, activation='relu')(GKInput)
DefLayer = layers.Dense(10, activation='relu')(DefInput)
ForLayer = layers.Dense(15, activation='relu')(ForInput)
DefMidLayer = layers.Dense(12, activation='relu')(DefMidInput)
MidLayer = layers.Dense(25, activation='relu')(MidInput)
RemLayer = layers.Dense(30, activation='relu')(RemInput)

merge = layers.concatenate([GKLayer, DefLayer, ForLayer, DefMidLayer, MidLayer, RemLayer])

hidden = layers.Dense(50, activation='relu')(merge)
# prediction output
output = layers.Dense(29, activation='softmax')(hidden)
model = models.Model(inputs=[GKInput,DefInput,ForInput,DefMidInput,MidInput, RemInput], outputs=output)

print(model.summary())
#plot_model(model, to_file='shared_input_layer.png')

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

train_yC = to_categorical(train_y)
test_yC = to_categorical(test_y)

best_weights = ModelCheckpoint('best.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='min')
callbacks_list = [best_weights]
history = model.fit([train_xGK, train_xDef, train_xFor, train_xDefMid, train_xMid, train_xRem], train_yC, epochs=10, batch_size=32,  callbacks=callbacks_list, verbose=0, validation_split=0.25)
model.load_weights("best.h5")#loads the best model saved wrt to validation loss during training

nn_pred_train= model.predict([train_xGK, train_xDef, train_xFor, train_xDefMid, train_xMid, train_xRem])
nn_pred_test= model.predict([test_xGK, test_xDef, test_xFor, test_xDefMid, test_xMid, test_xRem])

nn_pred_train = np.argmax(nn_pred_train,axis=1)
nn_pred_test = np.argmax(nn_pred_test,axis=1)

print ("NN Train Accuracy: ", metrics.accuracy_score(train_y, nn_pred_train))
print ("NN Test Accuracy: ", metrics.accuracy_score(test_y, nn_pred_test))
print ("NN Test MSE: ", metrics.mean_squared_error(test_y, nn_pred_test))